In [83]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import tkinter as tk
from tkinter import ttk
from ttkthemes import ThemedTk
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime


# Function

In [119]:
def creer_dossier_course():
    date_actuelle = datetime.now().strftime("%Y-%m-%d")
    nom_dossier = f"course_{date_actuelle}"
    chemin_dossier = os.path.join("C:\\Users\\Alex\\Desktop\\pmu_data", nom_dossier)
    try:
        os.mkdir(chemin_dossier)
        print(f"Dossier créé : {nom_dossier}")
    except FileExistsError:
        print(f"Le dossier {nom_dossier} existe déjà.")

def accept_coockies(driver:webdriver.Chrome):

    # J'acepte les coockies

    button_coockies = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "didomi-notice-agree-button")))
    button_coockies.click()

    #label.config(text="Le bouton des cookies a été cliqué!")
    return driver


def links_list_courses(driver:webdriver.Chrome,hrefs:list):
    # Trouver toutes les catégories de courses
    
    elements = driver.find_elements(By.CLASS_NAME, "pmu")
    for element in elements:
        link = element.find_element(By.TAG_NAME, "a")
        href = link.get_attribute("href")
        hrefs.append(href)
    return hrefs



def get_courses_data(driver):
    table_courses = driver.find_element(By.CSS_SELECTOR, "div.tableau table")
    header_row = table_courses.find_elements(By.TAG_NAME, "th")
    headers = [header.text for header in header_row]
    
    data = []
    rows = table_courses.find_elements(By.TAG_NAME, "tr")
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        row_data = [col.text for col in cols]
        part = int(row_data[4]) if row_data[4].isdigit() else 0
        if part == 10:
            course_link = cols[1].find_element(By.TAG_NAME, "a").get_attribute("href")
            data.append({
                'Nom de la course': row_data[1],
                'Lien': course_link,
                'Numéro': row_data[0]
            })

    return pd.DataFrame(data)


def create_df(driver:webdriver.Chrome,table_id):
    # Je mets les données dans un df

    table = driver.find_element(By.ID, table_id)
    header_row = table.find_elements(By.TAG_NAME, "th")
    headers = [header.text for header in header_row]
    data = []
    rows = table.find_elements(By.TAG_NAME, "tr")
    for row in rows[1:]: 
        cols = row.find_elements(By.TAG_NAME, "td")
        row_data = [col.text for col in cols]
        data.append(row_data)
    df = pd.DataFrame(data, columns=headers)
    return df

In [118]:
# Création dfes Excels

creer_dossier_course()
driver= webdriver.Chrome()
driver.get("https://www.zone-turf.fr/programmes")
driver = accept_coockies(driver)
hrefs = links_list_courses(driver,hrefs=[])
all_courses_df = pd.DataFrame()
for href in hrefs:
    driver.get(href)
    courses_df = get_courses_data(driver)
    all_courses_df = pd.concat([all_courses_df, courses_df], ignore_index=True)

date_actuelle = datetime.now().strftime("%Y-%m-%d")
nom_dossier = f"course_{date_actuelle}"
chemin_dossier = os.path.join("C:\\Users\\Alex\\Desktop\\pmu_data", nom_dossier)

for index, row in all_courses_df.iterrows():
    course_link = row['Lien']
    driver.get(course_link)
    

    numero = int(row['Numéro'])
    numero = numero - 1
    
    table_id = f"course{numero}"
    df_course_scrapper = create_df(driver,table_id)


    nom_fichier_excel = row['Nom de la course']
    file_name = f"{nom_fichier_excel}.xlsx"
    chemin = os.path.join(chemin_dossier,file_name)
    df_course_scrapper.to_excel(chemin, index=False)

driver.quit()

# Interface Tkinter

In [35]:
Path_fichier = ""

In [3]:
def on_button_click():
    label.config(text="Le bouton a été cliqué!")
def on_enter(event):
    button.config(bg="#4CAF50", fg="white")

def on_leave(event):
    button.config(bg="#388e3c", fg="white")

def on_click(event):
    button.config(bg="#1b5e20", fg="white")

def afficher_autres_boutons():
    bouton1 = tk.Button(root, text="Valider les courses")
    bouton1.pack(pady=5)


def afficher_cases_cocher():
    # Créer deux cases à cocher
    case1 = tk.Checkbutton(root, text="course1")
    case2 = tk.Checkbutton(root, text="course2")

    # Afficher les cases à cocher
    case1.pack(pady=5)
    case2.pack(pady=5)
    cases = [case1, case2]
    return cases

In [7]:

# La fenêtre principale

root = ThemedTk(theme="equilux")
root.title("Application Neunreuther")
root.geometry("400x300")

# Définir une couleur de fond pour la fenêtre
root.configure(bg="#f0f0f0")

# Créer une étiquette
label = tk.Label(root, text="Bienvenue dans mon application Papa", font=("Arial", 18), fg="#333333")
label.pack(pady=20)

# Créer un bouton
button = tk.Button(root, text="Cliquez ici pour scrapper les courses",command=lambda: (afficher_cases_cocher(), afficher_autres_boutons()),
                    font=("Arial", 12), bg="#388e3c", fg="white"
                  , borderwidth=0, border=5, relief="flat")



button.pack(pady=10, padx=20)  

button.bind("<Enter>", on_enter)
button.bind("<Leave>", on_leave)
button.bind("<Button-1>", on_click)


# Démarrer la boucle principale Tkinter
root.mainloop()